In [1]:
import pandas as pd
import yfinance as yf
import gspread
from gspread_dataframe import get_as_dataframe

In [4]:
def get_open_positions_from_gsheet(sheet_name='myportfolio', worksheet_name='portfolio'):
    gc = gspread.service_account(filename='gdrive-creds.json')
    sh = gc.open(sheet_name)
    ws = sh.worksheet(worksheet_name)
    df = get_as_dataframe(ws)
    df = df.dropna(subset=['Symbol'])  # Clean up empty rows

    # If your columns sometimes have unnamed first column, clean it:
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])

    # Keep only the latest DateTime for each Symbol
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.sort_values(['Symbol', 'DateTime'])
    latest_df = df.groupby('Symbol').tail(1).reset_index(drop=True)

    return latest_df


In [10]:
get_open_positions_from_gsheet(sheet_name='myportfolio', 
                               worksheet_name='portfolio').columns

Index(['DateTime', 'Symbol', 'Exchange', 'Currency', 'Position',
       'Market Price', 'Market Value', 'Average Cost', 'Unrealized PnL',
       'Realized PnL', 'Account'],
      dtype='object')

In [34]:
def get_latest_prices(symbols):
    price_dict = {}
    for symbol in symbols:
        try:
            data = yf.Ticker(symbol)
            latest = data.history(period="1d")['Close']
            if not latest.empty:
                price_dict[symbol] = latest.iloc[-1]
            else:
                price_dict[symbol] = None
        except Exception as e:
            price_dict[symbol] = None
    return price_dict


def get_latest_prices(symbols):
    price_dict = {}
    price_date_dict = {}
    for symbol in symbols:
        try:
            data = yf.Ticker(symbol)
            latest = data.history(period="1d")['Close']
            if not latest.empty:
                price_dict[symbol] = latest.iloc[-1]
                price_date_dict[symbol] = latest.index[-1].strftime('%Y-%m-%d')
            else:
                price_dict[symbol] = None
                price_date_dict[symbol] = None
        except Exception as e:
            price_dict[symbol] = None
            price_date_dict[symbol] = None
    return price_dict, price_date_dict


In [39]:
price_dict, price_date_dict = get_latest_prices(['IBKR'])
price_dates = list(price_date_dict.values())
unique_dates = set([d for d in price_dates if d is not None])
if len(unique_dates) == 1:
    price_date = unique_dates.pop()
else:
    price_date = "Mixed"


price_date

'2025-06-20'

{'IBKR': '2025-06-20'}

In [21]:
def calculate_yfinance_pnl(df, price_dict):
    """Given your positions DataFrame and a price_dict from get_latest_prices,
    calculate P&L columns."""
    results = []
    for _, row in df.iterrows():
        symbol = row['Symbol']
        qty = float(row['Position'])
        avg_cost = float(row['Average Cost'])
        last_price = price_dict.get(symbol)
        if last_price is not None:
            unrealized = (last_price - avg_cost) * qty
            pct = 100 * (last_price - avg_cost) / avg_cost if avg_cost != 0 else 0
            results.append({
                'Symbol': symbol,
                'Qty': qty,
                'Avg Cost': avg_cost,
                'Last Price': last_price,
                'Unrealized PnL': unrealized,
                'PnL %': pct
            })
    return pd.DataFrame(results)

# USAGE:
df = get_open_positions_from_gsheet()
price_dict = get_latest_prices(['IBKR'])
pnl_df = calculate_yfinance_pnl(df, price_dict)
# display(pnl_df)
pnl_df

,Symbol,Qty,Avg Cost,Last Price,Unrealized PnL,PnL %
0,IBKR,1.9276,51.706786,51.34,-0.707016,-0.709357


In [14]:
def summarize_portfolio(df):
    summary_lines = []
    summary_lines.append(f"Portfolio as of: {df['DateTime'].max()}\n")
    summary_lines.append("Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | Rlz PnL")
    summary_lines.append("-" * 58)
    for _, row in df.iterrows():
        summary_lines.append(
            f"{row['Symbol']:<7} | {row['Position']:>7.4f} | "
            f"{row['Average Cost']:>8.2f} | {row['Market Price']:>7.2f} | "
            f"{row['Unrealized PnL']:>9.2f} | {row['Realized PnL']:>7.2f}"
        )
    summary_lines.append("-" * 58)
    summary_lines.append(f"TOTAL Unrealized PnL: ${df['Unrealized PnL'].sum():.2f}")
    summary_lines.append(f"TOTAL Realized PnL:   ${df['Realized PnL'].sum():.2f}")
    return "\n".join(summary_lines)

if __name__ == "__main__":
    df = get_open_positions_from_gsheet()
    msg = summarize_portfolio(df)
    print(msg)


Portfolio as of: 2025-06-22 21:26:22

Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | Rlz PnL
----------------------------------------------------------
IBKR    |  1.9276 |    51.71 |   51.53 |     -0.34 |    0.00
----------------------------------------------------------
TOTAL Unrealized PnL: $-0.34
TOTAL Realized PnL:   $0.00


In [26]:
def summarize_yfinance_portfolio(pnl_df):
    summary_lines = []
    summary_lines.append("Portfolio (latest market prices)\n")
    summary_lines.append("Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | PnL %  ")
    summary_lines.append("-" * 60)
    for _, row in pnl_df.iterrows():
        summary_lines.append(
            f"{row['Symbol']:<7} | {row['Qty']:>7.4f} | "
            f"{row['Avg Cost']:>8.2f} | {row['Last Price']:>7.2f} | "
            f"{row['Unrealized PnL']:>9.2f} | {row['PnL %']:>6.2f}%"
        )
    summary_lines.append("-" * 60)
    total_unrlz_pnl = pnl_df['Unrealized PnL'].sum()
    total_invested = (pnl_df['Avg Cost'] * pnl_df['Qty']).sum()
    total_pnl_pct = 100 * total_unrlz_pnl / total_invested if total_invested != 0 else 0
    summary_lines.append(f"TOTAL Portfolio Unrealized P&L: ${total_unrlz_pnl:.2f}")
    summary_lines.append(f"TOTAL Portfolio Unrealised P&L %: {total_pnl_pct:.2f}%")
    return "\n".join(summary_lines)

# Example usage:
msg = summarize_yfinance_portfolio(pnl_df)
print(msg)


Portfolio (latest market prices)

Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | PnL %  
------------------------------------------------------------
IBKR    |  1.9276 |    51.71 |   51.34 |     -0.71 |  -0.71%
------------------------------------------------------------
TOTAL Portfolio Unrealized P&L: $-0.71
TOTAL Portfolio Unrealised P&L %: -0.71%


In [15]:
import requests

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"  # or "HTML"
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("Telegram message sent!")
    else:
        print("Failed to send:", response.text)


In [25]:
import requests

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"  # or "HTML"
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("Telegram message sent!")
    else:
        print("Failed to send:", response.text)

if __name__ == "__main__":
    # Your portfolio summary message
    msg = """
Portfolio as of: 2025-06-22 21:26:22

Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | Rlz PnL
----------------------------------------------------------
IBKR    |  1.9276 |    51.71 |   51.53 |     -0.34 |    0.00
----------------------------------------------------------
TOTAL Unrealized PnL: $-0.34
TOTAL Realized PnL:   $0.00
"""
    TELEGRAM_BOT_TOKEN = "7973671527:AAERsWVREF8c1cTjCNsrZ7003xHviwmCzWc"
    TELEGRAM_CHAT_ID = "7311840033"

    send_telegram_message(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, msg)


Telegram message sent!


In [27]:
def get_account_summary_from_gsheet(sheet_name='myportfolio', worksheet_name='accountsummary'):
    gc = gspread.service_account(filename='gdrive-creds.json')
    sh = gc.open(sheet_name)
    ws = sh.worksheet(worksheet_name)
    df = get_as_dataframe(ws)
    df = df.dropna(subset=['Tag'])
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.sort_values(['Tag', 'DateTime'])
    latest_df = df.groupby('Tag').tail(1).reset_index(drop=True)
    return latest_df


In [31]:
df = get_account_summary_from_gsheet(sheet_name='myportfolio', worksheet_name='accountsummary')

In [30]:
def summarize_account_summary(df):
    lines = []
    latest_datetime = df['DateTime'].max()
    lines.append(f"Account Summary as of: {latest_datetime}\n")
    lines.append("Tag                   | Value        | Currency")
    lines.append("-" * 45)
    for _, row in df.iterrows():
        lines.append(
            f"{row['Tag']:<22} | {row['Value']:>12} | {row['Currency']}"
        )
    lines.append("-" * 45)
    return "\n".join(lines)


In [33]:
print(summarize_account_summary(df))

Account Summary as of: 2025-06-22 21:26:22

Tag                   | Value        | Currency
---------------------------------------------
AvailableFunds         |       9965.0 | USD
BuyingPower            |       9965.0 | USD
NetLiquidation         |     10064.33 | USD
RealizedPnL            |          0.0 | BASE
TotalCashValue         |       9965.0 | USD
UnrealizedPnL          |        -0.34 | BASE
---------------------------------------------
